## Goal-Directed Molecule Generation with Reinforcement Learning: GCPN --> pLOGP
___
Goal-Directed Molecule Generation with Reinforcement Learning using GCPN.

This script showcases the application of the Graph Convolutional Policy Network (GCPN)
for goal-directed molecule generation on the ZINC250k dataset. The aim is to use reinforcement
learning to direct the model to generate molecules with specific properties. 
After loading the dataset and initializing the model, we employ the Proximal Policy Optimization (PPO)
algorithm as our RL criterion. The model is then fine-tuned, and the final results 
are displayed in the SMILES format.

Dependencies:
- torch
- torchdrug

Date: Sep.23.2023 \
Place: UC Merced

In [ ]:
import torch
from torchdrug import core, datasets, models, tasks
from torch import nn, optim
from collections import defaultdict

In [ ]:
dataset = datasets.ZINC250k(
    "~/molecule-datasets/", kekulize=True, atom_feature="symbol"
)

model = models.RGCN(
    input_dim=dataset.node_feature_dim,
    num_relation=dataset.num_bond_type,
    hidden_dims=[256, 256, 256, 256],
    batch_norm=False,
)
task = tasks.GCPNGeneration(
    model,
    dataset.atom_types,
    max_edge_unroll=12,
    max_node=38,
    task="plogp",
    criterion="ppo",
    reward_temperature=1,
    agent_update_interval=3,
    gamma=0.9,
)


In [ ]:
optimizer = optim.Adam(task.parameters(), lr=1e-5)
solver = core.Engine(
    task, dataset, None, None, optimizer, gpus=(0,), batch_size=16, log_interval=10
)

solver.load(
    "/home/akhanna2/data/git_stage/machine_learning_with_chemistry/torch_drug/molecule_generation/gcpn/graphgeneration/gcpn_zinc250k_1epoch.pkl",
    load_optimizer=False,
)

In [ ]:
# RL finetuning
solver.train(num_epoch=10)
solver.save(
    "/home/akhanna2/data/git_stage/machine_learning_with_chemistry/torch_drug/molecule_generation/gcpn/graphgeneration/gcpn_zinc250k_1epoch_finetune_pLOGP.pkl"
)

In [ ]:
results = task.generate(num_sample=32, max_resample=5)
print(results.to_smiles())
